In [1]:
from spacy import displacy
import spacy
from spacy.matcher import Matcher
from spacy.language import Language
import pandas as pd
import itertools

from namesRegex import TEST_SENTENCES, convertPenalties

nlp = spacy.load('en_core_web_sm')

In [2]:
matcher = Matcher(nlp.vocab)

patterns = [
    [{"POS": "PROPN"}, {"POS": "PROPN"}, {"ENT_TYPE": "PERSON"}],
    [{"POS": "PROPN"}, {"POS": "PROPN"}, {"POS": "PROPN"}, {"ENT_TYPE": "PERSON"}],
    [{"POS": "PROPN"}, {"POS": "PROPN"}],
    [
        {"POS": "PROPN", "OP": "+"},  # One or more proper nouns
        {"ORTH": "'", "OP": "?"},     # Optional apostrophe
        {"POS": "PROPN", "OP": "+"}   # One or more proper nouns
    ],
    [
        {"POS": "PROPN", "OP": "+"},   # One or more proper nouns
        {"TEXT": {"REGEX": "-"}, "OP": "?"},  # Optional hyphen
        {"POS": "PROPN", "OP": "+"}    # One or more proper nouns
    ],
    [
        {"POS": "PROPN"},            # Proper noun (first name)
        {"TEXT": {"REGEX": "'"}, "OP": "?"},  # Optional apostrophe
        {"POS": "PROPN"},            # Proper noun (last name)
        {"TEXT": {"REGEX": "-"}, "OP": "?"},  # Optional hyphen
        {"POS": "PROPN"},            # Proper noun (additional part of the last name, if any)
    ]
]
matcher.add("ALL_NAMES", patterns)

file = open('testDataCombo.txt', 'r')
names = file.read().split('\n')

empty_index = [i for i, name in enumerate(names) if len(name) == 0][0]

firstNames = names[:empty_index]
lastNames = names[empty_index + 1:]

combos = list(itertools.product(firstNames, lastNames))

names = [c[0] + " " + c[1] for c in combos]
    
for name in names:
    # print(name)
    for index, sentence in enumerate(TEST_SENTENCES):
        sentence = sentence.replace('@', name)
        sentence = convertPenalties(sentence)
        doc = nlp(sentence)
        names = [ent.text for ent in doc.ents if ent.label_ == "PERSON" and len(ent.text.split(" ")) > 1]
        matches = matcher(doc)
        for match_id, start, end in matches:
            string_id = nlp.vocab.strings[match_id]  # Get string representation
            span = doc[start:end]  # The matched span
            if span.text not in names:
                names.append(span.text)
        names = list(set(names))
        orig_names = names.copy()
        correct_names = names.copy()
        for i in range(len(names)):
            for j in range(len(names)):
                n0, n1 = names[i], names[j]
                if n1 in n0 and len(n1) < len(n0):
                    try:
                        correct_names.remove(n1)
                    except ValueError:
                        continue
        print(sentence)
        print(correct_names)
        # if len(correct_names) > 1:
        #     print(sentence)
        #     print(f"Original names: {orig_names}")
        #     print(correct_names)

Aaron Rodgers pass complete right to Aaron Rodgers for 16 yards (tackle by Aaron Rodgers and Aaron Rodgers)
['Aaron Rodgers']
Aaron Rodgers left for no gain. Aaron Rodgers fumbles, recovered by Aaron Rodgers. Aaron Rodgers fumbles, recovered by Aaron Rodgers and returned for 12 yards, touchdown, touchdown
['Aaron Rodgers']
Aaron Owusu-Koramoah pass complete right to Aaron Owusu-Koramoah for 16 yards (tackle by Aaron Owusu-Koramoah and Aaron Owusu-Koramoah)
['Aaron Owusu-Koramoah']
Aaron Owusu-Koramoah left for no gain. Aaron Owusu-Koramoah fumbles, recovered by Aaron Owusu-Koramoah. Aaron Owusu-Koramoah fumbles, recovered by Aaron Owusu-Koramoah and returned for 12 yards, touchdown, touchdown
['Aaron Owusu-Koramoah']
Aaron McCloud pass complete right to Aaron McCloud for 16 yards (tackle by Aaron McCloud and Aaron McCloud)
['Aaron McCloud']
Aaron McCloud left for no gain. Aaron McCloud fumbles, recovered by Aaron McCloud. Aaron McCloud fumbles, recovered by Aaron McCloud and returned f